# Setup Environment Variables (.env simulation)

In [1]:
import os

# Put your Gemini API key here
os.environ["GOOGLE_API_KEY"] = "YOUR_GEMINI_API_KEY"

# Optional: Langsmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = ""

# Gemini LLM + Embeddings

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

def get_llm():
    llm = ChatGoogleGenerativeAI(
        model="gemini-pro",
        temperature=0,
        max_output_tokens=512,
    )
    return llm

def get_embedding():
    embedding = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001"
    )
    return embedding

print("LLM + Embedding models ready.")


LLM + Embedding models ready.


# Documents for RAG

In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content=(
            "The T20 World Cup 2024 is in full swing, bringing excitement to cricket fans worldwide. "
            "India, led by Rohit Sharma, prepares for a crucial match against Ireland. "
            "Pitch conditions at the Nassau County stadium created controversy."
        ),
        metadata={"source": "cricket news"},
    ),
    Document(
        page_content=(
            "Football fans are excited as the UEFA Champions League semi-finals are set. "
            "Real Madrid faces Manchester City, while Bayern Munich plays Paris Saint-Germain."
        ),
        metadata={"source": "football news"},
    ),
    Document(
        page_content=(
            'The AI revolution continues to transform industries. '
            'AI-driven diagnostics in healthcare reduce costs and improve outcomes.'
        ),
        metadata={"source": "ai news"},
    )
]

print("Documents loaded:", len(documents))


Documents loaded: 3


# RAG Prompt Template

In [4]:
from langchain_core.prompts import ChatPromptTemplate

def get_rag_prompt():
    template = """
    Answer the question using ONLY the provided context.
    If the answer is not found in the context, say "I don't know".

    Question:
    {question}

    Context:
    {context}
    """
    prompt = ChatPromptTemplate.from_messages([("human", template)])
    return prompt

print("Prompt template ready.")


Prompt template ready.


# Chroma Vector DB Persistence

In [5]:
import os
from langchain_chroma import Chroma

def get_vectorstore(documents, embedding_model, persist_dir="./chroma_db"):
    os.makedirs(persist_dir, exist_ok=True)

    vectorstore = Chroma(
        persist_directory=persist_dir,
        embedding_function=embedding_model,
    )

    # CHECK: If DB empty → embed + store docs
    if not os.listdir(persist_dir):
        print("Vectorstore empty → adding documents...")
        vectorstore.add_documents(documents)
        vectorstore.persist()
    else:
        print("Loaded existing Chroma DB.")

    return vectorstore

print("Vectorstore loader ready.")


Vectorstore loader ready.


# Build RAG Pipeline

In [6]:
from langchain_core.runnables import RunnablePassthrough

def build_rag_chain(vectorstore, llm, prompt):
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 2},
    )
    
    chain = (
        {
            "context": retriever,
            "question": RunnablePassthrough()
        }
        | prompt
        | llm
    )
    
    return chain

print("RAG chain builder ready.")


RAG chain builder ready.


# Initialize Everything

In [7]:
llm = get_llm()
embedding_model = get_embedding()
vectorstore = get_vectorstore(documents, embedding_model)
prompt = get_rag_prompt()
rag_chain = build_rag_chain(vectorstore, llm, prompt)

print("All components initialized.")

Loaded existing Chroma DB.
All components initialized.


# Simple Function to Ask RAG a Question

In [9]:
def ask_rag(question: str):
    print("User:", question)
    response = rag_chain.invoke(question)
    answer = getattr(response, "content", str(response))  # safe extraction

    print("\nAI:", answer)
    return answer


# Test the RAG System

In [11]:
ask_rag("What is happening in the 2024 T20 world cup?")

User: What is happening in the 2024 T20 world cup?


GoogleGenerativeAIError: Error embedding content (INVALID_ARGUMENT): 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019b0d90-f654-7572-9bda-53c8d19eccae,id=019b0d90-f654-7572-9bda-53c8d19eccae; trace=019b0d90-f654-7572-9bda-53c8d19eccae,id=019b0d90-f671-7811-a84c-e2eaeeb653d7; trace=019b0d90-f654-7572-9bda-53c8d19eccae,id=019b0d90-f673-7e00-9548-e6bb54fdd6e6; trace=019b0d90-f654-7572-9bda-53c8d19eccae,id=019b0d90-f675-7262-a724-822299408e48; trace=019b0d90-f654-7572-9bda-53c8d19eccae,id=019b0d90-f675-7262-a724-822299408e48; trace=019b0d90-f654-7572-9bda-53c8d19eccae,id=019b0d90-f673-7e00-9548-e6bb54fdd6e6; trace=019b0d90-f654-7572-9bda-53c8d19eccae,id=019b0d90-f671-7811-a84c-e2eaeeb653d7; trace=019b0d90-f654-7572-9bda-53c8d19eccae,id=019b0d90-f654-7572-9bda-53c8d19eccae
